In [11]:
import pickle
from pathlib import Path
import numpy as np


from gfos.data.utils import load_layout
from gfos.metrics import LayoutMetrics
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"

In [4]:
files = load_layout(LAYOUT_DIR, compile_type="default", model_type="xla")

In [5]:
output_files = [
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_194701\multiruns\2023-10-25_19-47-01\0\wandb\run-20231025_194821-31cxqnvr\files\val_outs_359.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\0\wandb\run-20231025_212730-lylai3t4\files\val_outs_359.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\1\wandb\run-20231025_225204-bhxjen7t\files\val_outs_359.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\2\wandb\run-20231026_001619-w240tfbg\files\val_outs_199.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\3\wandb\run-20231026_011711-jphw5fx4\files\val_outs_239.plk",
]

In [8]:
outs = []
for file in output_files:
    with open(file, "rb") as f:
        data = pickle.load(f)
    outs.append(data)

In [14]:
metrics = LayoutMetrics()

for file in files["valid"]:
    model_id = Path(file).stem
    target = np.load(file)["config_runtime"]
    
    pred = []
    for out in outs:
        pred.append(out[model_id])
    pred = np.stack(pred).mean(axis=0)
    
    metrics.add(model_id, pred, target)

In [15]:
metrics.compute_scores()

{'raw_kendall': 0.44919012412011183,
 'index_kendall': 0.44917779441411504,
 'top100_error': 0.9500000000000001,
 'top500_error': 0.878857142857143,
 'kendall_bert_pretraining.4x4.fp16': 0.5236042774807833,
 'kendall_inception_v3_batch_128_train': 0.6507460755563222,
 'kendall_mlperf_bert_batch_24_2x2': 0.39715920177305986,
 'kendall_resnet50.4x4.fp16': 0.5626951090944631,
 'kendall_resnet_v1_50_official_batch_128_bf16': 0.43222062028529196,
 'kendall_tf2_bert_pretrain_dynamic_batch_size': 0.3796942475582187,
 'kendall_unet_3d.4x4.bf16': 0.19812502915066618}

In [16]:
test_files = [
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_194701\multiruns\2023-10-25_19-47-01\0\wandb\run-20231025_194821-31cxqnvr\files\xla_default_31cxqnvr_logits.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\0\wandb\run-20231025_212730-lylai3t4\files\xla_default_lylai3t4_logits.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\1\wandb\run-20231025_225204-bhxjen7t\files\xla_default_bhxjen7t_logits.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\2\wandb\run-20231026_001619-w240tfbg\files\xla_default_w240tfbg_logits.plk",
    r"G:\projects\gfos\logs\pipeline\layout_xla_default_20231025_212611\multiruns\2023-10-25_21-26-11\3\wandb\run-20231026_011711-jphw5fx4\files\xla_default_jphw5fx4_logits.plk",
]

test_outs = []
for file in test_files:
    with open(file, "rb") as f:
        data = pickle.load(f)
    test_outs.append(data)

In [23]:
results = {}

for file in files["test"]:
    model_id = Path(file).stem
    
    pred = []
    for out in test_outs:
        pred.append(out[model_id])
    pred = np.stack(pred).mean(axis=0)
    
    results[model_id] = pred


In [24]:
result_arg = {k: v.argsort().tolist() for k,v in results.items()}

In [26]:
SOURCE = "xla"
SEARCH = "default"


with open(f"../../output/{SOURCE}_{SEARCH}_blend_five_folds_1.csv", "w") as f:
    f.write("ID,TopConfigs\n")
    for k, v in result_arg.items():
        model_id = f"layout:{SOURCE}:{SEARCH}:"+k
        values = ";".join([str(i) for i in v])
        f.write(f"{model_id},{values}\n")